Loading the model in 

In [1]:
import torch
import pandas as pd
from torch import nn 

from common_utils import MLPBuilder
from torch.utils.data import DataLoader, TensorDataset


In [2]:
# we trained the model with 8 features
model = MLPBuilder(no_features=8, layers=[256,64,64])

model.load_state_dict(torch.load('models/NN_model.pth'))

C:\Users\randa\AppData\Local\Temp\ipykernel_18848\245742097.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/NN_model.pth'))


<All keys matched successfully>

## Loading test data in 

In [3]:
file_path = './Data/23_Dec_Readings/combined_data.csv'
cleaned_df = pd.read_csv(file_path)

In [4]:
cleaned_df

,Tx_0 RSSI,Tx_0 Batt,Tx_0 Time,Tx_1 RSSI,Tx_1 Batt,Tx_1 Time,Tx_2 RSSI,Tx_2 Batt,Tx_2 Time,Tx_3 RSSI,...,Tx_5 Batt,Tx_5 Time,Tx_6 RSSI,Tx_6 Batt,Tx_6 Time,Tx_7 RSSI,Tx_7 Batt,Tx_7 Time,X_Coord,Y_Coord
0,NaN,NaN,0.0,-77.0,4.11,0.0,-73.0,4.09,0.0,-75.0,...,4.05,0.0,-50.0,4.10,0.0,-75.0,NaN,0.0,7,0
1,NaN,NaN,24.0,-74.0,4.11,24.0,-73.0,NaN,28.0,-76.0,...,4.06,28.0,-49.0,4.10,29.0,-76.0,4.04,25.0,7,0
2,-82.0,NaN,53.0,-79.0,NaN,58.0,-73.0,4.09,58.0,-79.0,...,4.06,58.0,-49.0,4.11,60.0,-78.0,4.06,57.0,7,0
3,-82.0,4.11,81.0,-73.0,NaN,83.0,-73.0,4.09,84.0,-75.0,...,NaN,88.0,-49.0,4.09,90.0,-77.0,4.06,86.0,7,0
4,-81.0,NaN,111.0,-72.0,4.11,111.0,-72.0,NaN,115.0,-78.0,...,4.06,111.0,-48.0,4.11,111.0,-76.0,4.04,108.0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-75.0,NaN,390.0,-66.0,4.10,389.0,-81.0,4.08,383.0,-63.0,...,4.05,381.0,-74.0,4.11,382.0,-64.0,4.05,383.0,3,1
896,-76.0,NaN,414.0,-65.0,4.11,414.0,-81.0,4.09,408.0,-63.0,...,NaN,407.0,-75.0,4.10,408.0,-63.0,4.06,409.0,3,1
897,-77.0,4.11,436.0,-65.0,4.11,434.0,-79.0,4.09,429.0,-64.0,...,4.06,422.0,-76.0,4.10,421.0,-63.0,NaN,426.0,3,1
898,-77.0,4.11,453.0,-65.0,4.11,452.0,-81.0,NaN,451.0,-63.0,...,4.05,443.0,-76.0,4.11,443.0,-65.0,NaN,447.0,3,1


In [5]:
batt_columns = [col for col in cleaned_df.columns if 'Batt' in col]
time_columns = [col for col in cleaned_df.columns if 'Time' in col]
columns_to_drop = batt_columns+time_columns
RSSI_columns = cleaned_df.drop(columns=columns_to_drop)


#Train the model without NAN values
# cleaned_df = RSSI_columns.dropna()

#Convert NaN values to 0
cleaned_df = RSSI_columns.fillna(0)

In [6]:
cleaned_df

,Tx_0 RSSI,Tx_1 RSSI,Tx_2 RSSI,Tx_3 RSSI,Tx_4 RSSI,Tx_5 RSSI,Tx_6 RSSI,Tx_7 RSSI,X_Coord,Y_Coord
0,0.0,-77.0,-73.0,-75.0,-79.0,-63.0,-50.0,-75.0,7,0
1,0.0,-74.0,-73.0,-76.0,-78.0,-63.0,-49.0,-76.0,7,0
2,-82.0,-79.0,-73.0,-79.0,-77.0,-64.0,-49.0,-78.0,7,0
3,-82.0,-73.0,-73.0,-75.0,-80.0,-63.0,-49.0,-77.0,7,0
4,-81.0,-72.0,-72.0,-78.0,-81.0,-64.0,-48.0,-76.0,7,0
...,...,...,...,...,...,...,...,...,...,...
895,-75.0,-66.0,-81.0,-63.0,-67.0,-72.0,-74.0,-64.0,3,1
896,-76.0,-65.0,-81.0,-63.0,-67.0,-72.0,-75.0,-63.0,3,1
897,-77.0,-65.0,-79.0,-64.0,-66.0,-72.0,-76.0,-63.0,3,1
898,-77.0,-65.0,-81.0,-63.0,-66.0,-71.0,-76.0,-65.0,3,1


In [66]:
X = cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y = cleaned_df[['X_Coord', 'Y_Coord']]
X_test_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y.values, dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset) 



In [67]:
def evaluate_model(model, dataloader, loss_fn):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients during testing
        for inputs, targets in dataloader:
            # Move data to the appropriate device
            inputs, targets = inputs, targets

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            # Update running loss
            running_loss += loss.item()
            total += targets.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class index
            if targets.ndim == 1:  # Ensure targets are 1D
                correct += (predicted == targets).sum().item()
            else:
                correct += (predicted == targets.argmax(dim=1)).sum().item()

    # Compute average loss and accuracy
    avg_loss = running_loss / len(dataloader)
    accuracy = correct / total

    return avg_loss, accuracy

In [68]:
rssi_columns = ['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']

# Step 2: Flag blocked data (127 values) for each RSSI column
for col in rssi_columns:
    cleaned_df[col + '_blocked'] = (cleaned_df[col] == 127).astype(int)

# Step 3: Extract the feature data (X) and target data (Y)
X = cleaned_df[rssi_columns]  # Only the RSSI columns
X_with_flags = pd.concat([X, cleaned_df[[col + '_blocked' for col in rssi_columns]]], axis=1)  # Add the blocked flags
Y = cleaned_df[['X_Coord', 'Y_Coord']]  # Target coordinates

# Step 4: Perform the train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_with_flags, Y, test_size=0.2, random_state=42)



# Display the results
print("X_train:\n", X_train)
print("X_test:\n", X_test)
print("Y_train:\n", Y_train)
print("Y_test:\n", Y_test)

X_train:
      Tx_0 RSSI  Tx_1 RSSI  Tx_2 RSSI  Tx_3 RSSI  Tx_4 RSSI  Tx_5 RSSI  \
10         -81        -72        -74        -76        -81        -65   
334        -63        -65        -73        -70        -77        -64   
244        -84        -61        -75        -62        -80        -72   
678        -67        -75        -77        -79        -74        -75   
306        -58        -71        127        -75        -83        -76   
..         ...        ...        ...        ...        ...        ...   
106        -78        -70        -72        -82        -71        -54   
270        127        -60        -70        -72        -80        -57   
860        -73        -74        -68        -76        -67        -70   
435        -81        -74        -73        -65        -77        -68   
102        -79        -71        -73        -83        -71        -55   

     Tx_6 RSSI  Tx_7 RSSI  Tx_0 RSSI_blocked  Tx_1 RSSI_blocked  \
10         -50        -75                  0  

In [69]:
criterion = nn.MSELoss()
avg_loss, accuracy = evaluate_model(model,test_loader ,criterion)

print(f"Test Acc: {accuracy * 100:.2f}%, Test Loss: {avg_loss:.6f}\n")

Test Acc: 76.22%, Test Loss: 5.247501

